# Grouping scans by fieldmaps and phase encoding directions



In [1]:
%%bash

#SAMPLE_URL="https://upenn.box.com/shared/static/bvhs3sw2swdkdyekpjhnrhvz89x3k87t.xz"
#wget --retry-connrefused --waitretry=5 --read-timeout=20 --timeout=15 -t 0 -q \
# -O dscsdsi_buds.tar.xz ${SAMPLE_URL}
 
#tar xvfJ dscsdsi_buds.tar.xz
#ls DSCSDSI_BUDS
#rm dscsdsi_buds.tar.xz



In [2]:
from qsiprep.utils.bids import collect_data

def get_session_groups(layout, subject_data, combine_all_dwis):
    # Handle the grouping of multiple dwi files within a session
    sessions = layout.get_sessions() if layout is not None else []
    all_dwis = subject_data['dwi']
    dwi_session_groups = []
    if not combine_all_dwis:
        dwi_session_groups = [[dwi] for dwi in all_dwis]
    else:
        if sessions:
            print('Combining all dwi files within each available session:')
            for session in sessions:
                session_files = [img for img in all_dwis if 'ses-'+session in img]
                print('\t- %d scans in %s' % (len(session_files), session))
                dwi_session_groups.append(session_files)
        else:
            print(
                'Combining all %d dwis within the single available session' % len(all_dwis))
            dwi_session_groups = [all_dwis]
    return dwi_session_groups

subject_data, layout = collect_data("DSCSDSI_fmap_opts", "tester")
print(layout)

# Handle the grouping of multiple dwi files within a session
dwi_session_groups = get_session_groups(layout, subject_data, combine_all_dwis=True)

p1p2_group, = [group for group in dwi_session_groups if 'p1p2' in group[0]]
print(p1p2_group)

pdiff_group, = [group for group in dwi_session_groups if 'pdiff' in group[0]]
print(pdiff_group)

pepolar_group, = [group for group in dwi_session_groups if 'buds' in group[0]]
print(pepolar_group)

BIDS Layout: ...ep/notebooks/DSCSDSI_fmap_opts | Subjects: 1 | Sessions: 3 | Runs: 2
Combining all dwi files within each available session:
	- 2 scans in buds
	- 2 scans in p1p2
	- 2 scans in pdiff
['/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-p1p2/dwi/sub-tester_ses-p1p2_acq-HASC55_run-01_dwi.nii.gz', '/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-p1p2/dwi/sub-tester_ses-p1p2_acq-HASC55_run-02_dwi.nii.gz']
['/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-pdiff/dwi/sub-tester_ses-pdiff_acq-HASC55_run-01_dwi.nii.gz', '/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-pdiff/dwi/sub-tester_ses-pdiff_acq-HASC55_run-02_dwi.nii.gz']
['/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/dwi/sub-tester_ses-buds_acq-HASC55AP_dwi.nii.gz', '/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/dwi/sub-tester_ses-buds_acq-HASC5

Let's see how the layout gets the fieldmaps for each scan:

In [3]:
def list_fmaps(dwi_files):
    for dwi_file in dwi_files:
        fmaps = layout.get_fieldmap(dwi_file, return_list=True)
        print(dwi_file, '\n\t', fmaps, '\n')

print("PEPolar")
list_fmaps(pepolar_group)

print("\n\nphasediff")
list_fmaps(pdiff_group)


print("\n\np1p2")
list_fmaps(p1p2_group)


PEPolar
/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/dwi/sub-tester_ses-buds_acq-HASC55AP_dwi.nii.gz 
	 [{'epi': '/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/fmap/sub-tester_ses-buds_dir-PA_epi.nii.gz', 'type': 'epi'}] 

/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/dwi/sub-tester_ses-buds_acq-HASC55PA_dwi.nii.gz 
	 [{'epi': '/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/fmap/sub-tester_ses-buds_dir-AP_epi.nii.gz', 'type': 'epi'}] 



phasediff
/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-pdiff/dwi/sub-tester_ses-pdiff_acq-HASC55_run-01_dwi.nii.gz 
	 [{'phasediff': '/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-pdiff/fmap/sub-tester_ses-pdiff_phasediff.nii.gz', 'magnitude1': '/Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-pdiff/fmap/sub-tester_ses-pdiff_mag



```python
for dwi_session_group in dwi_session_groups:
    dwi_fmap_groups.extend(
        group_by_fieldmaps(dwi_session_group, layout,
                           "fieldmaps" in ignore or force_syn,
                           prefer_dedicated_fmaps,
                           combine_all_dwis))
```

Instead we will walk though the `group_by_fieldmaps` function to show how it works

In [4]:
from collections import defaultdict
import logging
LOGGER = logging.getLogger('nipype.workflow')
LOGGER.setLevel(logging.INFO)

def group_by_fieldmaps(dwi_files, layout, prefer_dedicated_fmaps, using_eddy,
                       combine_all_dwis):
    """Groups a session's DWI files by their PE direction and fieldmaps, if specified.

    DWIs are grouped by their **warped space**. Two DWI series that are
    listed in the IntendedFor field of a fieldmap are assumed to have the same
    susceptibility distortions and therefore be in the same warped space. The goal
    of this function is to combine DWI series into groups of acquisitions that
    are in the same warped space into a list of scans that can be combined after
    unwarping.

    The first step in the pipeline must be ``dwidenoise``. This algorithm can
    be run directly on each DWI Series. OR, DWI series in the same warped space
    can be combined before running ``dwidenoise``.  Running ``dwidenoise`` on
    concatenated DWI series that include noise and susceptibility distortion
    violates the assumptions made by ``dwidenoise`` and is not allowed.
    Performing susceptibility distortion correction before running ``dwidenoise``
    is also not allowed.

    Similarly, SHORELine performs head motion correction in **warped space**.
    The more q-space samples are available to SHORELine, the better the EAP
    estimates. However, if the position of the head is very different between
    the two runs it may take more iterations for SHORELine to converge. Be sure
    to check the QC files if using this algorithm.

    Specifying ``combine_all_dwis=True`` means that the SDC outputs will be
    used to combine across warped spaces. If ``False``, then no scans will be
    combined even if they are in the same warped space. Other
    possibilities for grouping are not supported.


    Parameters
    -----------

        dwi_files: list
            A list of full paths to dwi nifti files in a BIDS tree

        layout: BIDSLayout
            A representation of the BIDS tree

        prefer_dedicated_fmaps: bool
            If True, use files in fmap directories to perform SDC even if there
            is a series in dwi with the reverse phase encoding direction that
            could be used.

        combine_all_dwis: bool
            If True, concaenate DWI series that can be concatenated. To be concatenated
            the series must either have the same phase encoding direction or there
            must be fieldmaps that correct for opposite phase encoding directions.

    Returns:
    ---------

        groups_list: list
            A list of dictionaries that describe "group info". These can take a couple forms

            When there are two groups of dwi series with opposite phase encoding dirs:
            [
                {'dwi_series': ['scan1.nii', 'scan2.nii'],
                 'rpe_dwi_series': ['scan3.nii', 'scan4.nii'],
                 'rpe_b0': [],
                 'dwi_series_pedir': 'j'}
            ]

            When there is a PEPolar fieldmap
            [
             {
              'dwi_series': ['scan1.nii', 'scan2.nii'],
              'rpe_dwi_series': [],
              'rpe_b0: ['fmap_scan.nii'],
              'dwi_series_pedir': 'j'
             }
            ]

            When two different PE directions each have their own fieldmaps
            [
             {
              'dwi_series': ['scan1.nii', 'scan2.nii'],
              'rpe_dwi_series': [],
              'rpe_b0: ['fmap_scan.nii'],
              'dwi_series_pedir': 'j'
             },
             {
              'dwi_series': ['scan3.nii', 'scan4.nii'],
              'rpe_dwi_series': [],
              'rpe_b0: ['fmap_scan2.nii'],
              'dwi_series_pedir': 'j-'
             }
            ]

        NOTE: the pre-hmc steps, such as denoising, will only be run within warped space
        groups

    """
    rationale = ['The following DWI files were found with the following fieldmaps:']
    # How to prioritize fieldmaps?
    if prefer_dedicated_fmaps:
        fmap_priority = {
            'buds': 0,
            'epi': 1,
            'fieldmap': 2,
            'phasediff': 3,
            'phase1': 4,
            'syn': 5
        }
    else:
        fmap_priority = {
            'epi': 0,
            'fieldmap': 1,
            'phasediff': 2,
            'phase1': 3,
            'buds': 4,
            'syn': 5
        }

    # For doc-building
    if layout is None:
        LOGGER.warning("Assuming we're building docs")
        return [{'dwi_series': dwi_files,
                 'fieldmap_info':{'type': None},
                 'dwi_series_pedir': 'j',
                }]

    # For each DWI, figure out its PE dir and whether or not it was listed in
    # an IntendedFor field in a fieldmap sidecar. Make a list of
    # (filename, PE dir, fmap_file) and hang on to the pointers to additional files
    # for this fieldmap
    parsed_dwis = []
    fmap_pointers = {}
    for ref_file in dwi_files:
        metadata = layout.get_metadata(ref_file)

        # Find fieldmaps
        fmaps = layout.get_fieldmap(ref_file, return_list=True)
        fmap_file = ''
        priority = 99999
        for fmap in fmaps:
            fmap_type = fmap['type']
            if fmap_type not in ('epi', 'phasediff', 'phase'):
                continue

            if fmap_type == 'phase':
                fmap_type = 'phase1'

            this_priority = fmap_priority[fmap_type]
            if this_priority < priority:
                priority = this_priority
                fmap_file = fmap[fmap_type]
                fmap_pointers[fmap_file] = fmap
        
        fmap_tuple = (ref_file, metadata['PhaseEncodingDirection'], fmap_file, fmap_type)
        parsed_dwis.append(fmap_tuple)
        rationale.append(' * %s (PEDir %s): %s (Type %s)' % fmap_tuple)
        

    groups = []
    if not combine_all_dwis:
        rationale.append("\nEach DWI was assigned to its own group for preprocessing")
        for ref_file, pe_dir, fmap_file, fmap_type in parsed_dwis:
            groups.append({
                'dwi_series': [ref_file],
                'fieldmap_info': fmap_pointers[fmap_file],
                'dwi_series_pedir': pe_dir
            })
        print('\n'.join(rationale))
        return groups

    # group by warped space, defined by pe_dir and fmap
    groups = defaultdict(list)
    for ref_file, pe_dir, fmap, fmap_type in parsed_dwis:
        groups[(pe_dir, fmap)].append(ref_file)

    # Print the group memberships
    rationale.append("\nThe following warped spaces were defined to contain:")
    for warp_group, member_images in groups.items():
        rationale.append(' * PEDir: %s, fmap: %s ' % warp_group)
        for member_image in member_images:
            rationale.append('    -> ' + member_image)

    # group by pe dir and fmap
    complete_groups = []
    for warp_group, member_images in groups.items():
        pe_dir, fmap_file = warp_group
        complete_groups.append({
            'dwi_series': member_images,
            'fieldmap_info': fmap_pointers[fmap_file],
            'dwi_series_pedir': pe_dir
        })
    
    if prefer_dedicated_fmaps:
        rationale.append('\nStrictly using data from fmaps/ for SDC')
        print('\n'.join(rationale))
        return complete_groups

    # Ignoring dedicated fieldmaps, define warped spaces by PEDir
    match_header = 'No opportunities for bidirectional PEPolar DWI series were found.'
    pedir_groups = defaultdict(list)
    for warp_group, member_images in groups.items():
        pe_dir, _ = warp_group
        pedir_groups[pe_dir].extend(member_images)

    group_keys = set(list(pedir_groups.keys()))
    final_groups = []
    matches_rationale = []
    while len(group_keys):
        key = group_keys.pop()
        scan_list = pedir_groups[key]
        pe_axis = key[0]

        # Find opposing fieldmap groups
        matches = [k for k in group_keys if k[0] == pe_axis]
        num_matches = len(matches)

        if num_matches == 1:
            match_header = '\nThe following warp groups were used to SDC each other'
            # Found a perfect match
            match = matches[0]
            match_scan_list = pedir_groups[match]
            if using_eddy:
                matches_rationale.append('  * A single group containing %s/%s PE dir groups was sent to TOPUP' % (key, match))
                final_groups.append(
                    {'dwi_series': scan_list,
                     'fieldmap_info': {'type': 'rpe_series','rpe_series': match_scan_list,},
                     'dwi_series_pedir': pe_dir})
            else:
                matches_rationale.append('  * Two pipelines generated using %s/%s PE dir groups for SDC' % (key, match))
                final_groups.append(
                    {'dwi_series': scan_list,
                     'fieldmap_info': {'type': 'rpe_series', 'rpe_series': match_scan_list,},
                     'dwi_series_pedir': key})
                final_groups.append(
                    {'dwi_series': match_scan_list,
                     'fieldmap_info': {'type': 'rpe_series', 'rpe_series': scan_list},
                     'dwi_series_pedir': match})

            group_keys.discard(match)
        elif num_matches == 0:
            # Didn't find any reverse PE groups: add back all the original warp groups
            LOGGER.info("No matches found for %s", key)
                
            for warp_group, member_images in groups.items():
                pe_dir, fmap_file = warp_group
                if pe_dir == key:
                    final_groups.append({
                        'dwi_series': member_images,
                        'fieldmap_info': fmap_pointers[fmap_file],
                        'dwi_series_pedir': pe_dir
                    })
        else:
            # Didn't find any reverse PE groups
            LOGGER.critical("Multiple matches found for %s", key)
    
    print('\n'.join(rationale + [match_header] + matches_rationale))
    return final_groups


Separate the dwi groups into their fieldmap types to see how this function will perform on each

## Phase1/Phase2 fieldmaps

This PNC-style fieldmap will be useful to have, but is not currently implemented in main FMRIPREP.



In [5]:
import pprint
pp = pprint.PrettyPrinter(indent=2, width=40)
pp.pprint(
    group_by_fieldmaps(
        p1p2_group, layout, prefer_dedicated_fmaps=True, using_eddy=True, combine_all_dwis=True))

The following DWI files were found with the following fieldmaps:
 * /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-p1p2/dwi/sub-tester_ses-p1p2_acq-HASC55_run-01_dwi.nii.gz (PEDir j-): /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-p1p2/fmap/sub-tester_ses-p1p2_phase1.nii.gz (Type phase1)
 * /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-p1p2/dwi/sub-tester_ses-p1p2_acq-HASC55_run-02_dwi.nii.gz (PEDir j-): /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-p1p2/fmap/sub-tester_ses-p1p2_phase1.nii.gz (Type phase1)

The following warped spaces were defined to contain:
 * PEDir: j-, fmap: /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-p1p2/fmap/sub-tester_ses-p1p2_phase1.nii.gz 
    -> /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-p1p2/dwi/sub-tester_ses-p1p2_acq-HASC55_run-01_dwi.nii.gz
    -> /Users/mcieslak/projects/q

## PEPolar using eddy


### Use a dedicated RPE image from fmaps to correct each image
set ``prefer_dedicated_fmaps=True``

In [6]:
pp.pprint(
    group_by_fieldmaps(
        pepolar_group, layout, prefer_dedicated_fmaps=True, using_eddy=True, combine_all_dwis=True))

The following DWI files were found with the following fieldmaps:
 * /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/dwi/sub-tester_ses-buds_acq-HASC55AP_dwi.nii.gz (PEDir j): /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/fmap/sub-tester_ses-buds_dir-PA_epi.nii.gz (Type epi)
 * /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/dwi/sub-tester_ses-buds_acq-HASC55PA_dwi.nii.gz (PEDir j-): /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/fmap/sub-tester_ses-buds_dir-AP_epi.nii.gz (Type epi)

The following warped spaces were defined to contain:
 * PEDir: j, fmap: /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/fmap/sub-tester_ses-buds_dir-PA_epi.nii.gz 
    -> /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/dwi/sub-tester_ses-buds_acq-HASC55AP_dwi.nii.gz
 * PEDir: j-, fmap: /Users/mcieslak/projects

We see that two groupings are generated, each with an ``'epi'`` type fieldmap from the fmaps directory. 

### Using B0's from the RPE scan in the dwi directory

This is done by setting ``prefer_dedicated_fmaps=False``.

In [7]:
pp.pprint(
    group_by_fieldmaps(
        pepolar_group, layout, prefer_dedicated_fmaps=False, using_eddy=True, combine_all_dwis=True))

The following DWI files were found with the following fieldmaps:
 * /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/dwi/sub-tester_ses-buds_acq-HASC55AP_dwi.nii.gz (PEDir j): /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/fmap/sub-tester_ses-buds_dir-PA_epi.nii.gz (Type epi)
 * /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/dwi/sub-tester_ses-buds_acq-HASC55PA_dwi.nii.gz (PEDir j-): /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/fmap/sub-tester_ses-buds_dir-AP_epi.nii.gz (Type epi)

The following warped spaces were defined to contain:
 * PEDir: j, fmap: /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/fmap/sub-tester_ses-buds_dir-PA_epi.nii.gz 
    -> /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/dwi/sub-tester_ses-buds_acq-HASC55AP_dwi.nii.gz
 * PEDir: j-, fmap: /Users/mcieslak/projects

## PEPolar using SHORELine

### Use a dedicated RPE image from fmaps to correct each image
set ``prefer_dedicated_fmaps=True``

In [8]:
pp.pprint(
    group_by_fieldmaps(
        pepolar_group, layout, prefer_dedicated_fmaps=True, using_eddy=False, combine_all_dwis=True))

The following DWI files were found with the following fieldmaps:
 * /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/dwi/sub-tester_ses-buds_acq-HASC55AP_dwi.nii.gz (PEDir j): /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/fmap/sub-tester_ses-buds_dir-PA_epi.nii.gz (Type epi)
 * /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/dwi/sub-tester_ses-buds_acq-HASC55PA_dwi.nii.gz (PEDir j-): /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/fmap/sub-tester_ses-buds_dir-AP_epi.nii.gz (Type epi)

The following warped spaces were defined to contain:
 * PEDir: j, fmap: /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/fmap/sub-tester_ses-buds_dir-PA_epi.nii.gz 
    -> /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/dwi/sub-tester_ses-buds_acq-HASC55AP_dwi.nii.gz
 * PEDir: j-, fmap: /Users/mcieslak/projects

### Using B0's from the RPE scan in the dwi directory

This is done by setting ``prefer_dedicated_fmaps=False``.

In [9]:
pp.pprint(
    group_by_fieldmaps(
        pepolar_group, layout, prefer_dedicated_fmaps=False, using_eddy=False, combine_all_dwis=True))

The following DWI files were found with the following fieldmaps:
 * /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/dwi/sub-tester_ses-buds_acq-HASC55AP_dwi.nii.gz (PEDir j): /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/fmap/sub-tester_ses-buds_dir-PA_epi.nii.gz (Type epi)
 * /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/dwi/sub-tester_ses-buds_acq-HASC55PA_dwi.nii.gz (PEDir j-): /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/fmap/sub-tester_ses-buds_dir-AP_epi.nii.gz (Type epi)

The following warped spaces were defined to contain:
 * PEDir: j, fmap: /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/fmap/sub-tester_ses-buds_dir-PA_epi.nii.gz 
    -> /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-buds/dwi/sub-tester_ses-buds_acq-HASC55AP_dwi.nii.gz
 * PEDir: j-, fmap: /Users/mcieslak/projects

We see that for SHORELine there are two separate pipelines generated, each using the rpe series as the fieldmap. 

## Using phasediff fieldmaps

In [10]:
pp.pprint(
    group_by_fieldmaps(
        pdiff_group, layout, prefer_dedicated_fmaps=True, using_eddy=False, combine_all_dwis=True))

The following DWI files were found with the following fieldmaps:
 * /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-pdiff/dwi/sub-tester_ses-pdiff_acq-HASC55_run-01_dwi.nii.gz (PEDir j-): /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-pdiff/fmap/sub-tester_ses-pdiff_phasediff.nii.gz (Type phasediff)
 * /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-pdiff/dwi/sub-tester_ses-pdiff_acq-HASC55_run-02_dwi.nii.gz (PEDir j-): /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-pdiff/fmap/sub-tester_ses-pdiff_phasediff.nii.gz (Type phasediff)

The following warped spaces were defined to contain:
 * PEDir: j-, fmap: /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-pdiff/fmap/sub-tester_ses-pdiff_phasediff.nii.gz 
    -> /Users/mcieslak/projects/qsiprep/notebooks/DSCSDSI_fmap_opts/sub-tester/ses-pdiff/dwi/sub-tester_ses-pdiff_acq-HASC55_run-01_dwi.nii.gz
    ->